In [ ]:
%pip install faiss-cpu llama-index-vector-stores-faiss

In [1]:
# 라마인덱스 패키지 임포트
import faiss
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core import StorageContext
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

In [2]:
from dotenv import load_dotenv
import os

# .env 파일에서 환경 변수 로드
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
# LLM 설정(OpenAI)
Settings.llm = OpenAI(
    temperature=0.5,
    model="gpt-4o",
    max_tokens=200,
    context_window=4096,
)

In [5]:
# 데이터 로드 (data가 있는 경로 입력)
documents = SimpleDirectoryReader("../data/pdf_sample2").load_data()

In [6]:
# FAISS 인덱스 생성 (1536은 OpenAI 임베딩 차원)
dimension = 1536
faiss_index = faiss.IndexFlatL2(dimension)

In [7]:
# FAISS를 LlamaIndex의 인덱싱 및 검색 파이프라인에 통합
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# 문서 내용을 벡터 DB에 저장
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

2025-09-21 11:19:37,428 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [7]:
# (선택사항) FAISS 인덱스 저장
faiss.write_index(faiss_index, "faiss.index")

# (선택사항) FAISS 인덱스 로드
# loaded_index = faiss.read_index("faiss.index")

In [8]:
# 쿼리 엔진 생성
query_engine = index.as_query_engine()

# 쿼리 실행
query = "이 논문에서 제안하는 모델의 장점은 무엇인가요?"
response = query_engine.query(query)

# 응답 출력
print("\n질문:", query)
print("답변:", response)

2025-09-21 11:19:40,612 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-09-21 11:19:42,736 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



질문: 이 논문에서 제안하는 모델의 장점은 무엇인가요?
답변: 이 논문에서 제안하는 모델의 주요 장점은 효율적인 주의 메커니즘을 통해 번역 작업을 수행할 수 있다는 점입니다. 이 모델은 복잡한 구조 없이도 빠르고 효과적으로 문맥을 이해하고 번역할 수 있도록 설계되었습니다. 이를 통해 기존의 번역 모델보다 더 높은 성능을 발휘할 수 있습니다.
